## import the data from the database 

This is my attempt to make a connection between the postgres database and this file
It seems to work up until the import config from config portion 
I'm not sure if I don't have my environment set up correctly or what the issue is 
but it is needed because the program will recognize the class we are trying to use as 
a module


In [1]:
#import the password from a seperate file that is ignored on github
# from config import password
# import dependencies 
import psycopg2
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split

In [ ]:
from getpass import getpass 
password = getpass ('Enter database password')

try: 
    conn = psycopg2.connect(
        host="localhost",
        database = "Tennessee_Tornadoes",
        user="postgres",
        password=password)
    print ('pyscopg2 connection:', conn)
    
except Exception as err:
    print ('psycopg2 connect() ERROR:', err)
    connect = None


In [ ]:
cr = conn.cursor()
cr.execute('SELECT * FROM cleaned_tn_tornadoes;')
tmp = cr.fetchall()

In [ ]:
#extract the  column names
col_names = []
for db in cr.description:
    col_names.append(db[0])

In [ ]:
# Create the dataframe, passing in the list of col_names extracted from the description
df = pd.DataFrame(tmp, columns=col_names)
df.head()

In [ ]:
# #!/usr/bin/python
# from configparser import ConfigParser


# def config(filename='database.ini', section='postgresql'):
#     # create a parser
#     parser = ConfigParser()
#     # read config file
#     parser.read(filename)

#     # get section, default to postgresql
#     db = {}
#     if parser.has_section(section):
#         params = parser.items(section)
#         for param in params:
#             db[param[0]] = param[1]
#     else:
#         raise Exception('Section {0} not found in the {1} file'.format(section, filename))

#     return db


In [ ]:
# from config import config

In [ ]:
# #!/usr/bin/python

# def connect():
#     """ Connect to the PostgreSQL database server """
#     conn = None
#     try:
#         # read connection parameters
#         params = config()

#         # connect to the PostgreSQL server
#         print('Connecting to the PostgreSQL database...')
#         conn = psycopg2.connect(**params)

#         # create a cursor
#         cur = conn.cursor()
#     # execute a statement
#         print('PostgreSQL database version:')
#         cur.execute('SELECT version()')

#         # display the PostgreSQL database server version
#         db_version = cur.fetchone()
#         print(db_version)
       
#     # close the communication with the PostgreSQL
#         cur.close()
#     except (Exception, psycopg2.DatabaseError) as error:
#         print(error)
#     finally:
#         if conn is not None:
#             conn.close()
#             print('Database connection closed.')


# if __name__ == '__main__':
#     connect()
    
    


In [ ]:
#python connect.py

### Prepare Data for the ML

In [ ]:
# Create our features

target = ''
X = pd.get_dummies(df.drop(columns=target))


# Create our target
y = df[target]

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1)

### Run the first model

target = loss

In [ ]:
target = 'Property Loss'
X = pd.get_dummies(df.drop([target],axis = 1))


# Create our target
y = df[target]
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

 X_resampled, y_resampled = SMOTE(random_state=1,
    sampling_strategy='auto').fit_resample(
       X_resampled, y_resampled)
    Counter(y_resampled)
# Calculated the balanced accuracy score
    y_pred = model.predict(X_test)
    balanced_accuracy_score(y_test, y_pred)
    print(f"The accuracy score: \n{balanced_accuracy_score(y_test, y_pred)}\n")
# Display the confusion matrix

### Run the second model 

target = location 

In [ ]:


target = 'starting county'
X = pd.get_dummies(df.drop([target],axis = 1))


# Create our target
y = df[target]
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

model = make_pipeline(StandardScaler(),SGDClassifier(max_iter=1000, tol=1e-3))
model.fit(X_resampled, y_resampled)

# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)
print(f"The accuracy score: \n{balanced_accuracy_score(y_test, y_pred)}\n")


### Run the third model 

target = Magnitude 

In [ ]:
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)
# Train the Logistic Regression model using the resampled data

model = LogisticRegression(solver='liblinear', random_state=1)
    
#'liblinear’, ‘sag’, ‘saga
model.fit(X_resampled, y_resampled)
# Display the confusion matrix

y_pred = model.predict(X_test)
print(f"The accuracy score: \n{balanced_accuracy_score(y_test, y_pred)}\n")
